# ИССЛЕДОВАНИЕ НАДЕЖНОСТИ ЗАЕМЩИКОВ

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Данные включают в себя следующую информацию о каждом клиенте: количество детей в семье, общий трудовой стаж, возраст, уровень образования, семейное положение, пол, тип занятости, наличие задолженности по возврату кредитов, ежемесячный доход и цель получения кредита.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# План проекта

## Изучение набора данных
## Предобработка данных
    2.1. Обработка пропусков
    2.2. Замена типа данных
    2.3. Обработка дупликатов
    2.4. Лемматизация
    2.5. Категоризация данных
## Установление зависимостей между категориями данных
## Общий вывод

## 1 Изучение набора данных

In [2]:
#импортируем библиотеку pandas и получим общую информацию по таблице
import pandas as pd
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#Выведем несколько строк таблицы, чтобы посмотреть, что из себя представляют сами данные
data.loc[8:13]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи
10,2,-4171.483647,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.491460,покупка недвижимости
11,0,-792.701887,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
13,0,-1846.641941,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля


**Вывод** Каждая строка таблицы содержит определенную информацию о клиенте банка. Для решения поставленной задачи наиболее важны столбцы children, family status, debt. В начале нужно решить пробелмы с пропусками и дубликатами.

## 2 Предобработка данных

### Обработка пропусков

In [4]:
#посчитаем количество пропусков с помощью метода isnull
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод** Обнаружились пропуски в двух столбцах - общий трудовой стаж и ежемесячный доход. Причинами пропуска в доходах может быть их нестабильность, человек может быть не устроен, тогда его стаж не подсчитывается и доход тоже является неофициальным. возможно человек относится к нетрудоспособному населению, неработающий студент или в принципе безработный.
Чтобы заполнить пропуски в столбце с ежемесячным доходом подсчитаем средний уровень дохода по каждому типу занятости. 
Трудовой стаж не играет особой роли для ответа на посталенные вопросы, поэтому можем оставить пропуски в данном столбце.

In [5]:
#посмотрим все уникальные виды занятости
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Заменим пропуски в столбце total_income на средний доход по каждому типу занятости:

In [6]:
#сгруппируем данные по типу занятости и заполним пропуски в столбце "ежемесячный доход" медианным значением из соответствующей группы. 
#выведем на экран часть таблицы и проверим, что пропуски заполнены
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median())).round()   
data[['income_type', 'total_income']].head(15)

,income_type,total_income
0,сотрудник,253876.0
1,сотрудник,112080.0
2,сотрудник,145886.0
3,сотрудник,267629.0
4,пенсионер,158616.0
5,компаньон,255764.0
6,компаньон,240526.0
7,сотрудник,135824.0
8,сотрудник,95857.0
9,сотрудник,144426.0


### Замена типа данных

Ежемесячный доход представлен вещественныыми числами, было бы удобнее видеть их целочисленными.

In [7]:
data['total_income'] = data['total_income'].values.astype(int)

Так же в столбце с количеством детей всплывает "-1", понятно, что такого быть не может, скорее всего это опечатка и минус лишний. Заменим "-1" на "1", точно так же "20" заменим на "2", здесь тоже скорее всего опечатка и ноль лишний.

In [8]:
#заменим опечатки на нужные значения методом replace и проверим с помощью метода unique, какое количество детей повторяется в данных
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Обработка дубликатов

Ищем дубликаты:

In [9]:
#посчитаем количество дупликатов с помощью метода duplicated
data.duplicated().sum()

54

In [10]:
#удалим дупликаты и выполним проверку
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

В начале исследования при выводе части таблицы видим, что в столбце "образование" разный регистр. Приведем данные к единому виду.

In [11]:
#приведем данные в столбце с образованием к нижнему регистру
#проверим правильность выполнения выводом нескольких первых строк
data['education'] = data['education'].str.lower()
data['education'].head(10)

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
5     высшее
6     высшее
7    среднее
8     высшее
9    среднее
Name: education, dtype: object

Снова проверим наличие дубликатов.

In [12]:
#вызываем метод duplicated
data.duplicated().sum()

17

После изменения регистра появлись новые дубликаты, удалим их.

In [13]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод** Дубликаты могли появиться в результате какого-то технологического сбоя или человеческого фактора, возможно кто-то подал заявку дважды.

### Лемматизация

In [14]:
#посмотрим все уникальные цели по столбцу с целью кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Видно, что многие цели дублируются, объединим их в категории.

In [15]:
#для этого сначала разобьем на отдельные слова строки в столбце с целями с помощью лемматизации
from pymystem3 import Mystem
m = Mystem()

def credit_purpose(row):
    lemmas = m.lemmatize(row['purpose'])
    return ''.join(lemmas)
data['credit_purpose'] = data.apply(credit_purpose, axis=1)
data['credit_purpose'].head(10)

0                 покупка жилье\n
1       приобретение автомобиль\n
2                 покупка жилье\n
3    дополнительный образование\n
4               сыграть свадьба\n
5                 покупка жилье\n
6              операция с жилье\n
7                   образование\n
8         на проведение свадьба\n
9       покупка жилье для семья\n
Name: credit_purpose, dtype: object

In [16]:
#выделим несколько категорий: недвижимость, авто, образование, прочее, и согласно этому изменим столбец с целями
def purpose_grouped(row):
    lem=row['credit_purpose']
    if 'жилье' in lem or 'недвижимость' in lem:
        return 'недвижимость'
    elif 'автомобиль' in lem:
        return 'автомобиль'
    elif 'образование' in lem:
        return 'образование'
    else:
        return 'прочее'
data['credit_purpose'] = data.apply(purpose_grouped, axis=1)
print(data['credit_purpose'].head(10))

0    недвижимость
1      автомобиль
2    недвижимость
3     образование
4          прочее
5    недвижимость
6    недвижимость
7     образование
8          прочее
9    недвижимость
Name: credit_purpose, dtype: object


**Вывод** При изучении столбца с целями кредита заметили, что одни и те же цели имеют разные формулировки, мы упростили столбец, выделив несколько основных категорий. С помощью лемматизации мы выделили отдельные слова в ячейках столбца "цель", среди них выделили "маркеры" для каждой категории, с помощью которых сформировали новый столбец с целями кредитования, в котором всего 4 уникальных значения. В категорию "недвижимость" я объединила все, что связано с недвижимостью - покупка, строительство, ремонт, какаие-то другие операции (это крупная категория, ее можно было бы разделить на коммерческую недвижимость, жилую недвижимость и мелкие операции с недвижимостью(например, ремонт), но я посчитала, что для запроса данного исследования это не столь важно), в категорию "автомобиль" попало все, что связано с автомобилем - покупка, сделка, новый или поддержанный, в категорию "образование" попало все образование - дополнительное, высшее, профильное, и категория "прочее", в нашем случае туда попали только заемщики, желающие провести свадьбу. 

### Категоризация данных

 В предыдущем шаге мы получили некий словарь, содержащий категории целей кредитования. Мы выделили недвижимость, атомобиль, образование и прочее (например, заем на свадьбу).

**Вывод** Категоризация столбца с целями кредитования помогла сделать таблицу компактей и избавиться от дубликатов.

## 3 Установление зависимостей между категориями данных

Выясним, есть ли зависимость между наличием детей и возвратом кредита в срок.

In [17]:
#сделаем сводную таблицу с количеством детей и суммарным количеством заемщиков с просрочками 
data_children = data.pivot_table(index=['children'], values='debt', aggfunc='sum')
data_children

,debt
children,
0,1063
1,445
2,202
3,27
4,4
5,0


По полученному результату можно сказать, что чем больше детей - тем меньше вероятность задолженности по кредиту, но это будет некорректно, поэтому посчитаем отношение количества задолженников к заемщикам с каждым количеством детей и приведем его в процентах

In [18]:
#считаем количество заемщиков для каждого количества детей
data_children['all'] = data.groupby('children')['debt'].count()
data_children

,debt,all
children,,
0,1063,14091
1,445,4855
2,202,2128
3,27,330
4,4,41
5,0,9


In [19]:
#рассчитаем процент должников для каждого количества детей
data_children['percent_of_debt'] = ((data_children['debt'] / data_children['all']) * 100).round(1)
data_children

,debt,all,percent_of_debt
children,,,
0,1063,14091,7.5
1,445,4855,9.2
2,202,2128,9.5
3,27,330,8.2
4,4,41,9.8
5,0,9,0.0


**Вывод** По полученным значениям можно заметить, что для заемщиков без детей процент задолженности меньше, значение для заемщиков с 5 детьми считаем выбросами. Это объясняется тем, что дети - это источник непрогнозируемых трат.

Выясним, есть ли зависимость между семейным положением и возвратом кредита в срок.

In [20]:
#просмотрим уникальные семейные статусы
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [21]:
#сделаем сводную таблицу с суммарным количеством должников для каждого семейного статуса
data_family_status = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
data_family_status

,debt
family_status,
Не женат / не замужем,274
в разводе,85
вдовец / вдова,63
гражданский брак,388
женат / замужем,931


In [22]:
#посчитаем общее количество заемщиков для каждого семейного статуса
data_family_status['all'] = data.groupby('family_status')['debt'].count()
data_family_status

,debt,all
family_status,,
Не женат / не замужем,274,2810
в разводе,85,1195
вдовец / вдова,63,959
гражданский брак,388,4151
женат / замужем,931,12339


In [23]:
#посчитаем процент должников для каждого семейного статуса
data_family_status['percent_of_debt'] = ((data_family_status['debt'] / data_family_status['all']) * 100).round(1)
data_family_status

,debt,all,percent_of_debt
family_status,,,
Не женат / не замужем,274,2810,9.8
в разводе,85,1195,7.1
вдовец / вдова,63,959,6.6
гражданский брак,388,4151,9.3
женат / замужем,931,12339,7.5


**Вывод** По полученным значениям можно заметить, что процент вероятности задолженности выше у неженатых/незамужних и живущих гражданским браком заемщиков. Это может объснятся тем, что 

Выясним, есть ли зависимость между уровнем дохода и возвратом кредита в срок.

In [24]:
#Найдем величину максимального дохода
data['total_income'].max()

2265604

In [25]:
#Найдем величину минимального дохода
data['total_income'].min()

20667

In [26]:
#Найдем среднее значение дохода
data['total_income'].mean()

165320.05243777385

Выделим 3 группы по уровню дохода: до 100 000, от 100 000 до 500 000 и более 500 000.

In [27]:
#Напишем функцию, которая каждому значению дохода присвоит категорию: "низкий", "средний" или "высокий"
#Выведем несколько строк для проверки результата
def income_level_group(row):
    if row['total_income'] <= 100000:
        return 'низкий'
    elif row['total_income'] > 500000:
        return 'высокий'
    else:
        return 'средний'
data['income_level'] = data.apply(income_level_group, axis=1)
data['income_level'].loc[47:51]

47    средний
48    средний
49     низкий
50     низкий
51    средний
Name: income_level, dtype: object

In [28]:
#составим сводную таблицу по уровням дохода и суммарным количеством должником для каждого из них
data_total_income = data.pivot_table(index=['income_level'], values='debt', aggfunc='sum')
data_total_income

,debt
income_level,
высокий,14
низкий,354
средний,1373


In [29]:
#посчитаем общее количество заемщиков с каждым уровнем дохода
data_total_income['all'] = data.groupby('income_level')['debt'].count()
data_total_income

,debt,all
income_level,,
высокий,14,222
низкий,354,4463
средний,1373,16769


In [30]:
#посчитаем процент должников с каждым уровнем дохода
data_total_income['percent_of_debt'] = ((data_total_income['debt'] / data_total_income['all']) * 100).round(1)
data_total_income

,debt,all,percent_of_debt
income_level,,,
высокий,14,222,6.3
низкий,354,4463,7.9
средний,1373,16769,8.2


**Вывод** По полученным значениям видно, что нет явной зависимости между уровнем дохода и способностью погашать кредиты в срок. Можно отметить, что бОльший процент должников среди клиентов со средним доходом. Вероятно это объясняется, что клиенты с низким доходом "не замахиваются" на большие суммы, а клиенты с высоким доходом не имеют финансовых проблем и чаще погашают кредит в срок.

Рассмотрим, как разные цели кредита влияют на его возврат в срок.

In [31]:
#составим сводную таблицу по целям кредитования и количеством должников
data_credit_purpose = data.pivot_table(index=['credit_purpose'], values='debt', aggfunc='sum')
data_credit_purpose

,debt
credit_purpose,
автомобиль,403
недвижимость,782
образование,370
прочее,186


In [32]:
#посчитаем количество заемщиков по каждой цели кредитования
data_credit_purpose['all'] = data.groupby('credit_purpose')['debt'].count()
data_credit_purpose

,debt,all
credit_purpose,,
автомобиль,403,4306
недвижимость,782,10811
образование,370,4013
прочее,186,2324


In [33]:
#посчитаем процент должников по каждой цели кредитования
data_credit_purpose['percent_of_debt'] = ((data_credit_purpose['debt'] / data_credit_purpose['all']) * 100).round(1)
data_credit_purpose

,debt,all,percent_of_debt
credit_purpose,,,
автомобиль,403,4306,9.4
недвижимость,782,10811,7.2
образование,370,4013,9.2
прочее,186,2324,8.0


**Вывод** Можно сделать вывод, что люди подходят более осознанно и ответственно к большим заемам, таким как недвижимость, этим объясняется относительно низкий процент задолженности по этим кредитам

## 4 Выводы

Мы провели анализ данных статистики о платежеспособности клиентов банка. 
- Были обнаружены пропуски и дубликаты, от которых мы успешно избавились, так же обнаружили опечатки в столбце с количеством детей и поправили их, опираясь на жизненную логику.
- Сделали столбец с целями кредита по категориям: недвижимость, автомобиль, образование и прочее.

Далее мы провели оценку данных по некоторым характеристикам заемщиков и наличию задолженности по предыдущим кредитам, сделали следующие выводы:
- наличие детей увеличивает вероятность возникновения задолженности;
- неженатые/незамужние и живущие гражданским браком чаще не возвращают заем в срок;
- заемщики с высоким доходом реже становятся должниками;
- кредиты на недвижимость реже остальных попадают в просроченные.

По основному вопросу от заказчика можно сказать, что вне зависимости от семейного положения, наличие детей у заемщиков увеличивает риск появления задолженности по кредиту, верно и обратное, вне зависимотсти от количества детей у клиентов, находящихся или ранее бывших в официальных семейных отношениях, реже случаются задолженности по кредиту.

Считаю этих двух категорий недостаточно, чтобы оценить риск возникновения задолженности. По проведенному анализу видно что уровень дохода сильно влияет на возникновение задолженности(например, если у клиента высокий доход и 5 детей, это не так сильно повышает риск задолженности, как низкий и нестабильный доход у заемщика). Так же цель кредитования тоже важна, мы отметили, что задолженности по кредитам на недвижимость случаются реже, так например собственное жилье - это более важный и ответственный аспект жизни, чем автомобиль. Эти категории так же нужно принимать во внимание. Так же мы везде опираемся на наличие задолженности в прошлом, но клиент может брать кредит впервые, соответственно и задолженности не было, этой информации не хватает, ее тоже неплохо было бы учитывать.